<a href="https://colab.research.google.com/github/PrasunaA/LangChain1/blob/main/LangChain_Demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import Generation, LLMResult
from langchain_core.pydantic_v1 import Field
from typing import Optional, List, Any

class HuggingFacePipelineLLM(LLM):
    pipeline: Any = Field(...)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        result = self.pipeline(prompt, max_length=256, do_sample=True)
        return result[0]["generated_text"]

    def _generate(self, prompt: str, stop: Optional[List[str]] = None) -> LLMResult:
        output_text = self._call(prompt, stop)
        return LLMResult(generations=[[Generation(text=output_text)]])

    @property
    def _llm_type(self) -> str:
        return "custom-hf-pipeline"


In [ ]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-large")
wrapped_llm = HuggingFacePipelineLLM(pipeline=generator)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = LLMChain(llm=wrapped_llm, prompt=prompt)

response = chain.invoke({"topic": "AI agents"})
print(response["text"])
